In [1]:
# вимикаємо зайві попередження
import warnings
warnings.filterwarnings("ignore")

# друк всіх результатів в одній комірці а не тільки останнього
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# імпорт додаткових модулей
from dataclasses import dataclass, field

In [6]:
# Визначення класу Teacher
@dataclass(order=True)
class Teacher:
	"""Клас для опису вчиталя."""
	first_name: str = field(default=None, compare=False)
	last_name: str = field(default=None, compare=False)
	age: int = field(default=None, compare=False)
	email: str = field(default=None, compare=False)
	can_teach_subjects: set = field(default_factory=set, compare=False)

	# Поля, які не потрібно ініціалізувати при створенні об'єкта та не потрібно виводити в репрезентації.
	# їх призначення - бути полями для сортування за кількістю покритих предметів та молодістю.
	# "Найбільший" при порівнянні викладач - найбільш приорітетний за кількістю покритих предметів та молодістю
	number_of_current_covered_subjects: int = field(default=0, init=False, repr=False) # містить кількість предметів, які покриває вчитель з поточної ітерації відбору
	age_for_sorting: int = field(default=None, init=False, repr=False) # службове поле - містить вік вчителя з протилежним знаком, щоб сортувати за віком від старшого до молодшого

	def __post_init__(self):
		self.age_for_sorting = - self.age

	def __str__(self):
		return f"Вчитель {self.first_name} {self.last_name}, {self.age} років, email: {self.email}, предмети: {', '.join(self.can_teach_subjects)}"
	
	def calculate_priority(self, uncovered_elements):
		"""Заповнює поле сортування кількістю предметів, які викладає вчитель на поточному етапі відбору."""
		self.number_of_current_covered_subjects = len(self.can_teach_subjects & uncovered_elements)

In [7]:
# функція жадібного алгоритму для покриття множини
def create_schedule(subjects, teachers):
    
   # Зберігатиме відібраних вчителів
    chosen_teacher = []
    
    # Копія непокритих предметів для того, щоб не змінювати початкову множину
    uncovered_elements = subjects.copy()

    # Поки є непокриті елементи та вільні вчителі
    while teachers and uncovered_elements:
        
		# розраховуємо кількість покритих елементів для кожного вчителя для поточного набору непокритих предметів
        for teacher in teachers:
            teacher.calculate_priority(uncovered_elements)
        
        # Знайдемо вчителя, який покриває найбільшу кількість непокритих елементів, якщо таких декілька, то виберемо наймолодшого
        best_teacher = max(teachers)
        covered_subjects = best_teacher.can_teach_subjects
        
        # Переносимо найкращого в цьому циклі вчителя до списку обраних
        chosen_teacher.append(teachers.pop(teachers.index(best_teacher)))
        
        # Видаляємо покриті елементи з множини непокритих
        uncovered_elements -= covered_subjects
        
	# Виводимо інформацію про обраних вчителів, вчителів без покритих предметів та непокриті предмети
    return chosen_teacher, teachers, uncovered_elements

In [8]:
if __name__ == '__main__':
    
    # Множина предметів
    subjects = {'Математика', 'Фізика', 'Хімія', 'Інформатика', 'Біологія'}
    
    # Створення списку викладачів
    teachers = [
			Teacher("Олександр", "Іваненко", 45, 'o.ivanenko@example.com', {"Математика", "Фізика"}),
			Teacher("Марія", "Петренко", 38, 'm.petrenko@example.com', {"Хімія",}),
			Teacher("Сергій", "Коваленко", 50, 's.kovalenko@example.com', {"Інформатика", "Математика"}),
			Teacher("Наталія", "Шевченко", 29, 'n.shevchenko@example.com', {"Біологія", "Хімія"}),
			Teacher("Дмитро", "Бондаренко", 35, "d.bondarenko@example.com", {"Фізика", "Інформатика"}),
			Teacher("Олена", "Гриценко", 42, "o.grytsenko@example.com", {"Біологія",}),
			]
    
    # Виклик функції створення розкладу
    schedule = create_schedule(subjects, teachers)

    # Виведення розкладу
    if schedule[0]:
        print("Розклад занять:\n======================")
        for teacher in schedule[0]:
            print(f"{teacher.first_name} {teacher.last_name}, {teacher.age} років, email: {teacher.email}")
            print(f"   Викладає предмети: {', '.join(teacher.can_teach_subjects)}\n")
        print("\nНепокриті предмети:\n======================")
        for subject in schedule[2]:
            print(f"   {subject}")
        print('\nВільні викладачі:\n======================')
        for teacher in schedule[1]:
            print(f"{teacher.first_name} {teacher.last_name}, {teacher.age} років, email: {teacher.email}")
            print(f"   Викладає предмети: {', '.join(teacher.can_teach_subjects)}\n")
    else:
        print("Неможливо покрити всі предмети наявними викладачами.")

Розклад занять:
Наталія Шевченко, 29 років, email: n.shevchenko@example.com
   Викладає предмети: Біологія, Хімія

Дмитро Бондаренко, 35 років, email: d.bondarenko@example.com
   Викладає предмети: Інформатика, Фізика

Олександр Іваненко, 45 років, email: o.ivanenko@example.com
   Викладає предмети: Математика, Фізика


Непокриті предмети:

Вільні викладачі:
Марія Петренко, 38 років, email: m.petrenko@example.com
   Викладає предмети: Хімія

Сергій Коваленко, 50 років, email: s.kovalenko@example.com
   Викладає предмети: Інформатика, Математика

Олена Гриценко, 42 років, email: o.grytsenko@example.com
   Викладає предмети: Біологія

